### 任务一_填充证券代码列

In [ ]:
import pandas as pd

# 从指定路径读取Excel文件
file_path = r'E:\Work-Script\data pre processing\审计五个需求\data.xlsx'

# 读取Excel文件内容到DataFrame
df = pd.read_excel(file_path)

# 使用groupby和transform填充缺失的'证券代码1'，填充值为同一公司全称下的第一个'证券代码1'的值
df['证券代码1'] = df.groupby('公司全称')['证券代码1'].transform('first')

# 打印填充后的DataFrame
print(df)

# 如果需要，将填充后的数据保存回Excel文件中
df.to_excel('filled_data.xlsx', index=False)

### 任务二_数据(针对2008年)的清洗去重

In [ ]:
# 从指定路径读取Excel文件
file_path = r'E:\Work-Script\data pre processing\审计五个需求\filled_data.xlsx'

# 读取Excel文件
df = pd.read_excel(file_path)

# 检查是否会计截止日期连续重复
consecutive_duplicates = df['会计截止日期'].eq(df['会计截止日期'].shift(-1))

# 删除标记为True的行
df = df[~consecutive_duplicates]

# 打印处理后的数据框
print(df)

# 保存到新的Excel文件
df.to_excel('delete_2008.xlsx', index=False)

### 任务三_意见分类

In [ ]:
# 从指定路径读取Excel文件
file_path = r'E:\Work-Script\data pre processing\审计五个需求\delete_2008.xlsx'

# 读取Excel文件
df = pd.read_excel(file_path)

# 创建一个映射，将'审计意见类型'映射到0和1
opinion_mapping = {
    '标准无保留意见': 0,
    '保留意见': 1,
    '保留意见加事项段': 1,
    '否定意见': 1,
    '没有单位': 1,
    '无保留意见加事项段': 1,
    '无发表意见': 1
}

# 使用map函数将映射应用到'审计意见类型'列，并将结果赋值给'上期审计意见'列
df['上期审计意见'] = df['审计意见类型'].map(opinion_mapping)

# 打印处理后的数据框
print(df)

# 保存到新的Excel文件
df.to_excel('opinion_categorization.xlsx', index=False)

### 任务四_计算业务复杂度

In [ ]:
# 从指定路径读取Excel文件
file_path = r'E:\Work-Script\data pre processing\审计五个需求\业务复杂度.xlsx'

# 读取Excel文件
df = pd.read_excel(file_path)

# 删除包含空值的行
df = df.dropna(subset=['应收款项融资', '存货净额', '资产总计'])

# 如果有其他复杂的计算公式，请替换下面的代码
df['业务复杂度'] = df['应收款项融资'] + df['存货净额'] + df['资产总计']

# 打印处理后的数据框
print(df)

# 保存到新的Excel文件
df.to_excel('service_complexity.xlsx', index=False)


In [33]:
import pandas as pd

# 定义源文件和目标文件的路径
src_path = r'service_complexity.xlsx'
dst_path = r'opinion_categorization.xlsx'

# 读取源文件和目标文件
df_source = pd.read_excel(src_path)
df_target = pd.read_excel(dst_path)

# 将'证券代码'和'证券代码1'列转换为字符串类型，以确保数据类型一致
df_source['证券代码'] = df_source['证券代码'].astype(str)
df_target['证券代码1'] = df_target['证券代码1'].astype(str)

# 确保列名匹配，这里假设源文件的'证券代码'和'统计截止日期'分别对应目标文件的'证券代码1'和'会计截止日期'
df_source = df_source.rename(columns={'证券代码': '证券代码1', '统计截止日期': '会计截止日期'})

# 合并数据
# 根据证券代码和日期合并，这里使用inner join，只有当两个文件都有匹配的代码和日期时才合并
df_merged = pd.merge(df_target, df_source[['证券代码1', '会计截止日期', '业务复杂度']], on=['证券代码1', '会计截止日期'], how='inner')

# 保存到新的Excel文件
df_merged.to_excel('merged_file.xlsx', index=False)


### 任务五_计算经营现金流比例和销售增长率

### 任务六_日期整理